In [79]:
import os
import re
import json
from sklearn.preprocessing import MultiLabelBinarizer 
import itertools
import pandas as pd
import numpy as np
import webbrowser
from datetime import date

In [84]:
#--- basic cleaning ---#
def empty_list_to_string(x):
    if (isinstance(x,list) and len(x)==0):
        return ""
    else:
        return x
    
def remove_unit_signs(x):
    return re.sub("\s*m²|\s*€","",x)

def remove_thousand_dot(x):
    return re.sub("\.(?=[0-9]{3})","",x)

def replace_commas(x):
    return re.sub(",(?=[0-9]{1,2})",".",x)

def remove_rows_without_price_or_zip(df):
    tmpDf = df[["preis","plz"]]
    nrowComplete = df.shape[0]
    mask = tmpDf.preis.notna() & (df.plz!="")
    nrowClean = mask.sum()
    df = df.loc[mask]
    print(nrowComplete-nrowClean," rows without price and zip data were deleted")
    return df

def clean_df(df):
    df = df.applymap(empty_list_to_string)
    unitSignsList = ["preis", "wohnflaeche","grundstuecksflaeche"]
    df[unitSignsList] = (df[unitSignsList]
                         .applymap(remove_unit_signs)
                         .applymap(remove_thousand_dot)
                         )
    return df

#--- clean zip and place ---#
def get_zip_and_place(df):
    def funk(x):
        try:
            x = re.findall("\d{5}",x)[0] 
        except:
            x = ""
        return x
    df["plz"] = df.ort.apply(funk)

    df["ortsname"] = df.ort.apply(lambda x: re.sub("\d{5}","",x) if len(x)>0 else x)

    df = df.drop(["ort"],axis=1) 
    return df

#--- get binarized columns of characteristics ---#
def clean_merkmale(ser):
    ser = ser.apply(lambda x: re.sub('^, ','',x) if x is not None else x)#comma at the start
    ser = ser.apply(lambda x: re.sub(',$','',x) if x is not None else x)# comma at the end

    ser = ser.apply(lambda x: x.split(',') if x is not None else x) 
    ser = ser.apply(lambda x: [re.sub('^\s','',a) for a in x] if x is not None else x)#eliminate whitespace
    ser = ser.apply(lambda x: ["Keine Angabe"] if len(x)==1 else x)#set value for empty list
    print("merkmale cleaned and put into list")
    return ser

def binarize_merkmale(df):
    mlb = MultiLabelBinarizer()
    df["merkmale"] = clean_merkmale(df["merkmale"])
    dfMerkmale = pd.DataFrame(mlb.fit_transform(df.merkmale)
                             ,columns=mlb.classes_
                             ,index=df.index)
    df = pd.concat([df,dfMerkmale],axis=1)
    df = df.drop(['merkmale'],axis=1)
    print("merkmale binarized")
    return df

def set_key_value_as_index(df):
    df["id"] = df.url.apply(lambda x: re.findall(r"\w+",x)[-1])
    df = df.set_index("id",drop=True)
    return df

def drop_unprepared(df):
    """
    drops columns containing secondary information that have not yet been prepared
    """
    df = df.drop(["title","url","weitere_eigenschaften","beschreibung"],axis=1)
    return df



def prepare_price(df):
    """
    replaces commas with dots, replaces "auf Anfrage" (=on request) with empty string, 
    strips leading or trailing blanks and converts to float64
    
    """
    if df.preis.dtypes != "float64":
        tmp_preis = df.preis
        tmp_preis = tmp_preis.str.replace('auf Anfrage\xa0','')
        tmp_preis = tmp_preis.str.strip()
        tmp_preis = tmp_preis.apply(replace_commas)
        df["preis"] = pd.to_numeric(tmp_preis)
        print("preis has been cleaned up and converted to float")
    else: 
        print("preis is already float")
    return df

def clean_numerical_cols(df):
    numCols = ["anzahl_raeume","wohnflaeche","grundstuecksflaeche"]
    df[numCols] = df[numCols].applymap(replace_commas).applymap(pd.to_numeric)
    print("numerical columns:",', '.join(numCols),"cleaned")
    return df

def unpack_list_elements(df):
    """
    some columns' elements are lists. This is incompatible with eg drop_duplicates().
    This file unpacks them as str
    """
    listCols = ["weitere_eigenschaften","beschreibung"]
    df[listCols] = df[listCols].applymap(lambda x:", ".join(x))
    return df

# --- loading --- #

def extract_metadata_from_filename(jsonFileName):
    tmpDictCols = {}
    tmp = jsonFileName.split("-")
    tmpDictCols["transaktionsArt"] = tmp[-1].split(".")[0]
    tmpDictCols["objektArt"] = tmp[-2]
    tmpDictCols["datumDownload"] = "-".join(tmp[:3])
    tmpDictCols["suchOrt"] = "-".join(tmp[3:-2])
    return tmpDictCols

def insert_meta_data_columns(dfTmp,jsonFileName):
    """
    The json files do not contain metadata. They are read from the filename,
    prepared by extract_meta_data_from_filename and put into columns at the 
    start of the dataframe
    
    input:
        - df 
        - jsonFileName (str)
        
    output:
        - df
    """
    tmpDictCols = extract_metadata_from_filename(jsonFileName)
    for colNum, name in enumerate(tmpDictCols.keys()):
        dfTmp.insert(loc=colNum, column=name, value = tmpDictCols[name])
    return dfTmp

def get_pathdata_and_listfilenames(location="notebook"):
    """
    creates path to raw data in json format, and creates a generator with
    [path]/[filename]
    
    Paramaters:
        location (str): "notebook" if this file is run from notebook folder, 
                        if any other string if run from main folder of project.
    
    Returns:
        pathFile (generator)
    """
    pathData = os.path.join("..","data")
    jsonFilesList = [a for a in os.listdir(pathData) if re.findall("json$",a)]
    for fileName in jsonFilesList:
        pathFile = os.path.abspath(os.path.join(pathData,fileName))
        yield pathFile, fileName



def load_data(location="notebook"):
    """
    loads the data in data folder as json into pandas and adds metadata columns 
    derived from filenames
    
        Parameters:
            location (str): "notebook" if this file is run from notebook folder, 
                            if any other string if run from main folder of project.
            
        Results:
            df
    """
    
    dfList = []
    for pathFile, nameFile in get_pathdata_and_listfilenames():
        #print(nameFile)
        with open(pathFile) as data_file:    
            data = json.load(data_file)
        dfTmp = pd.json_normalize(data,['objects'])
        dfTmp = insert_meta_data_columns(dfTmp,nameFile)
        dfList.append(dfTmp)
    print(len(dfList),f' json-files have been loaded')
    df = pd.concat(dfList)
   
    return df

def load_and_prepare_data():
    df = load_data()
    df = set_key_value_as_index(df)

    df = clean_df(df)
    df = get_zip_and_place(df)
    df = remove_rows_without_price_or_zip(df)

    df = binarize_merkmale(df)
    df = unpack_list_elements(df)
    df = prepare_price(df)
    df["datumDownload"] = pd.to_datetime(df.datumDownload)    
    df = clean_numerical_cols(df)
    df = df.drop_duplicates()
    
    return df


In [85]:
df = load_and_prepare_data()

33  json-files have been loaded
373  rows without price and zip data were deleted
merkmale cleaned and put into list
merkmale binarized
preis has been cleaned up and converted to float
numerical columns: anzahl_raeume, wohnflaeche, grundstuecksflaeche cleaned


In [94]:
df[(df.preis.isna()==0)&(df.suchOrt=="norderstedt")].sort_values(by="datumDownload",ascending=False)

,transaktionsArt,objektArt,datumDownload,suchOrt,url,title,preis,anzahl_raeume,wohnflaeche,grundstuecksflaeche,...,frei,provisionsfrei,renoviert,rollstuhlgerecht,saniert,seniorengerechtes Wohnen,teilweise klimatisiert,teilweise unterkellert,voll klimatisiert,voll unterkellert
id,,,,,,,,,,,,,,,,,,,,,
2xblv4d,mieten,wohnungen,2021-04-13,norderstedt,https://www.immowelt.de/expose/2xblv4d,Erstbezug! Moderne Erdgeschosswohnung mit Gart...,1120.0,3.0,95.94,95.94,...,0,0,0,0,0,0,0,0,0,0
2ztzz43,kaufen,wohnungen,2021-04-13,norderstedt,https://www.immowelt.de/expose/2ztzz43,WO IHNEN GANZ EIMSBÜTTEL ZU FÜSSEN LIEGT - EIN...,2639000.0,4.0,195.40,195.40,...,0,0,0,0,0,0,0,0,0,0
2xlzm4z,kaufen,wohnungen,2021-04-13,norderstedt,https://www.immowelt.de/expose/2xlzm4z,BESTE AUSSICHTEN FÜR STILVOLLES WOHNEN Etagenw...,899000.0,4.0,124.00,124.00,...,0,0,0,0,0,0,0,0,0,0
2xe5n4z,kaufen,wohnungen,2021-04-13,norderstedt,https://www.immowelt.de/expose/2xe5n4z,RESERVIERT | BESTE AUSSICHTEN FÜR STILVOLLES W...,1169000.0,4.0,151.50,151.50,...,0,0,0,0,0,0,0,0,0,0
2xwjn4z,kaufen,wohnungen,2021-04-13,norderstedt,https://www.immowelt.de/expose/2xwjn4z,BESTE AUSSICHTEN FÜR STILVOLLES WOHNEN - Penth...,1100000.0,4.0,129.10,129.10,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2yl2a48,kaufen,haus,2021-03-09,norderstedt,https://www.immowelt.de/expose/2yl2a48,Wohnen & Arbeiten unter einem Dach - Großzügig...,589000.0,8.0,200.98,940.00,...,0,0,0,0,0,0,0,0,0,0
2wzyt4q,kaufen,haus,2021-03-09,norderstedt,https://www.immowelt.de/expose/2wzyt4q,Die schönsten Stadthäuser Hamburgs Doppelhaush...,3250000.0,5.0,233.00,450.00,...,0,0,0,0,0,0,0,0,0,0
2yfrc47,kaufen,haus,2021-03-09,norderstedt,https://www.immowelt.de/expose/2yfrc47,Einzigartiges Friesenhaus mit Einliegerwohnung...,1200000.0,7.0,210.00,1554.00,...,0,0,0,0,0,0,0,0,0,1


In [75]:
interessanteCols = ["preis","anzahl_raeume","wohnflaeche","transaktionsArt","url","datumDownload","plz"]


In [77]:
mask = ((df["anzahl_raeume"]>=2) 
        & (df["wohnflaeche"]>10) 
        #& (df["transaktionsArt"]=="mieten") 
        & (df["preis"]<1500)
        #& (df["datumDownload"]=="2021-04-24")#pd.to_datetime('today').normalize())
        #& (df["plz"]!="24568")
       )
(df.loc[mask,interessanteCols].sort_values(by=["datumDownload"],ascending=False))

,preis,anzahl_raeume,wohnflaeche,transaktionsArt,url,datumDownload,plz
id,,,,,,,


In [69]:
mask = ((df["anzahl_raeume"]>=1) 
        & (df["wohnflaeche"]>10) 
        & (df["transaktionsArt"]=="kaufen") 
        & (df["preis"]<500000)
        & (df["datumDownload"]==pd.to_datetime('today').normalize())
       # & (df["plz"]!="24568")
        #& (df["plz"]!="29493")
       )
subsel = (df.loc[mask,interessanteCols]
 .sort_values(by=["preis","anzahl_raeume"])
)
subsel

,preis,anzahl_raeume,wohnflaeche,transaktionsArt,url,datumDownload,plz
id,,,,,,,


In [70]:

firefoxPath = "/Applications/Firefox.app/Contents/MacOS"
client = webbrowser.get("open -a /Applications/Firefox.app %s")
for url in subsel.url[5:10]:
    client.open(url)